In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# /kaggle/input/car-crash-dataset-ccd/CrashBest
# /kaggle/input/car-crash-dataset-ccd/CrashBest/C_000001_01.jpg
# /kaggle/input/crash-1500/Crash-1500/000001.mp4
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!sudo apt-get update
!sudo apt-get install ffmpeg

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:6 https://packages.cloud.google.com/apt google-fast-socket InRelease [1071 B]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:8 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [24.8 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1490 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1559 kB]
Get:12 https://packages.cloud.google.com/apt cloud-sdk/main amd64 P

In [2]:
import pandas as pd
from sklearn.utils import resample

csv_file_path = '/kaggle/input/preprocessed-df/preprocessed_df (1).csv'
annotations = pd.read_csv(csv_file_path)

class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
Moderate    622
Minor       348
Severe      244
moderate     70
severe       57
minor        31
fatal        25
Fatal        18
Name: count, dtype: int64


In [3]:
annotations['Severity of the Crash'] = annotations['Severity of the Crash'].str.lower()
class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
moderate    692
minor       379
severe      301
fatal        43
Name: count, dtype: int64


In [4]:
annotations['Severity of the Crash'] = annotations['Severity of the Crash'].str.replace('fatal', 'severe', case=False)
annotations['label'] = annotations['label'].replace(3, 2)
class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
moderate    692
minor       379
severe      344
Name: count, dtype: int64


In [5]:
df_minor = annotations[annotations['Severity of the Crash'] == 'minor']
df_moderate = annotations[annotations['Severity of the Crash'] == 'moderate']
df_severe = annotations[annotations['Severity of the Crash'] == 'severe']

In [6]:
max_count = max(class_counts)
df_minor_oversampled = resample(df_minor, replace=True, n_samples=max_count, random_state=21)
df_severe_oversampled = resample(df_severe, replace=True, n_samples=max_count, random_state=21)
annotations_oversampled = pd.concat([df_minor_oversampled, df_moderate, df_severe_oversampled])

# Shuffle the dataset
annotations_oversampled = annotations_oversampled.sample(frac=1, random_state=21).reset_index(drop=True)

# Updated class counts after oversampling
print(annotations_oversampled['Severity of the Crash'].value_counts())

Severity of the Crash
severe      692
minor       692
moderate    692
Name: count, dtype: int64


In [ ]:
import os
import pandas as pd

video_dir = '/kaggle/input/crash-1500/Crash-1500'
frames_dir = '/kaggle/working/frames' 

for idx, row in annotations.iterrows():
    video_number = row['Video Number']
    video_path = os.path.join(video_dir, f'{video_number:06}.mp4')
    output_dir = os.path.join(frames_dir, f'{video_number:06}')
    os.makedirs(output_dir, exist_ok=True)

    ffmpeg_command = f'ffmpeg -i "{video_path}" -vf fps=4 "{output_dir}/frame%04d.png"'
#     print(f"Running FFmpeg command: {ffmpeg_command}")
    os.system(ffmpeg_command)
    
    extracted_frames = os.listdir(output_dir)
    print(f"Extracted {len(extracted_frames)} frames for video {video_number}")

In [7]:
from PIL import ImageFilter,ImageEnhance
import cv2
import numpy as np

def denoise_frame(frame):
    frame = np.array(frame)
    denoised_frame = cv2.medianBlur(frame, 5)
    return Image.fromarray(denoised_frame)

def deblur_frame(frame):
    return frame.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))

def enhance_contrast(frame):
    frame = np.array(frame)
    lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    enhanced_frame = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    return Image.fromarray(enhanced_frame)

def sharpen_frame(frame, factor=2.0):
    enhancer = ImageEnhance.Sharpness(frame)
    return enhancer.enhance(factor)

In [8]:
import os
frames_dir = '/kaggle/working/frames'
video_dirs = next(os.walk(frames_dir))[1]
num_videos = len(video_dirs)

print(f"Number of videos with extracted frames: {num_videos}")

Number of videos with extracted frames: 1415


In [9]:
import os
import glob
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image

class VideoDataset(Dataset):
    def __init__(self, annotations_df, root_dir, transform=None, num_frames=20):
        self.annotations = annotations_df
        self.annotations['Severity of the Crash'] = self.annotations['Severity of the Crash'].str.lower()
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {'minor': 0, 'moderate': 1, 'severe': 2}
        self.num_frames = num_frames

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        video_number = self.annotations.iloc[idx]['Video Number']
        label = self.label_map[self.annotations.iloc[idx]['Severity of the Crash']]
        frame_dir = os.path.join(self.root_dir, f'{video_number:06}')
        frames = []
        # print(f"Looking for frames in: {frame_dir}")
        for frame_path in sorted(glob.glob(os.path.join(frame_dir, '*.png'))):
            frame = read_image(frame_path)
            frame = transforms.ToPILImage()(frame)
            frame = denoise_frame(frame)
            frame = sharpen_frame(frame)
            frame = enhance_contrast(frame)
            frame = deblur_frame(frame)
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)
        # print(f"Found {len(frames)} frames.")

        if not frames:  # Check if frames list is empty
            raise ValueError(f"No frames found for video {video_number} in directory {frame_dir}")
        frames = torch.stack(frames)
        return frames, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(10), 
    transforms.ToTensor(),  
])

dataset = VideoDataset(annotations_df=annotations_oversampled, root_dir='/kaggle/working/frames', transform=transform)

In [10]:
import torch
from transformers import VideoMAEConfig, VideoMAEModel
import torch.nn as nn
import torch.optim as optim

In [11]:
class VideoClassifier(nn.Module):
    def __init__(self, num_classes, num_frames):
        super().__init__()
        self.config = VideoMAEConfig(
            num_frames=num_frames,
            num_classes=num_classes
        )
        self.videomae = VideoMAEModel(self.config)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

    def forward(self, x):
        outputs = self.videomae(x)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits

model = VideoClassifier(num_classes=3, num_frames=20)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

VideoClassifier(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x VideoMAELayer(
          (attention): VideoMAESdpaAttention(
            (attention): VideoMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (dense): Linear(in_

In [ ]:
class_counts = annotations['label'].value_counts()
print(class_counts)

In [12]:
import numpy as np
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler, SubsetRandomSampler
from collections import Counter

np.random.seed(21)
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

train_split = int(np.floor(0.7 * dataset_size))
val_split = int(np.floor(0.2 * dataset_size))
test_split = dataset_size - train_split - val_split

train_indices = indices[:train_split]
val_indices = indices[train_split:train_split + val_split]
test_indices = indices[train_split + val_split:]

print(f"Total dataset size: {dataset_size}")
print(f"Training set size: {len(train_indices)}")
print(f"Validation set size: {len(val_indices)}")
print(f"Test set size: {len(test_indices)}")

train_labels = [annotations_oversampled['Severity of the Crash'][idx] for idx in train_indices]
class_counts = Counter(train_labels)
total_samples = len(train_labels)
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
print(class_weights)
sample_weights = [class_weights[annotations_oversampled['Severity of the Crash'][idx]] for idx in train_indices]
sample_weights = torch.tensor(sample_weights, dtype=torch.float)

train_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=4, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=2, sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=2, sampler=test_sampler)

print("Total effective samples in training set:", len(train_loader) * train_loader.batch_size)

Total dataset size: 2076
Training set size: 1453
Validation set size: 415
Test set size: 208
{'moderate': 2.9653061224489794, 'minor': 2.9592668024439917, 'severe': 3.0783898305084745}
Total effective samples in training set: 1456


In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 4
best_val_acc = 0.0  
save_path = '/kaggle/working/best_model.pth'  

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in train_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Here the model is evaluated on the validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total

    print('[%d] loss: %.3f, val_acc: %.3f' % (epoch + 1, running_loss / len(train_loader), val_acc))
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"New best model saved with val_acc: {best_val_acc:.3f}")

[1] loss: 3.019, val_acc: 33.494
New best model saved with val_acc: 33.494
[2] loss: 1.245, val_acc: 36.145
New best model saved with val_acc: 36.145
[3] loss: 1.231, val_acc: 37.831
New best model saved with val_acc: 37.831
[4] loss: 1.288, val_acc: 33.253


In [14]:
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = 100 * correct / total
print('Test Accuracy: %.3f' % test_acc)

Test Accuracy: 26.442


In [15]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=73f0e5bc67921fe5db43d6a2886e6033bccae49f3d89a6b018d50154542374e7
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 98% |
|  1 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 | 100% |  1% |
|  1 |   0% |  0% |
